In [48]:
import os
os.getcwd()

'/home'

In [49]:
os.chdir("..")
import utils.functions as fc
import utils.indicators as indic

In [50]:
from importlib import reload
fc = reload(fc)
indic = reload(indic)

In [51]:
ship_data_enriched = fc.create_ship_data_enriched()

In [52]:
ais = fc.read_ais_parquet()

In [53]:
ais.shape

(4406590, 6)

In [54]:
ais_enriched = fc.enrich_AIS_data(ais, ship_data_enriched)

In [18]:
#ais_enriched = fc.filter_cargo(ais_enriched)

In [11]:
ais_enriched.shape

(2634715, 125)

In [55]:
# function qui assigne le pays d'origine
ais_enriched2, origin_countries = indic.assign_country_origin(ais_enriched)

/home/onyxia/work/hackathon-un-2022/utils/indicators.py:10: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  mmsi_number = pd.read_csv(fs.open(f'{BUCKET}/AIS/mmid.csv',


Classification of origin country
Russian Federation                                                                   387
Republic of Türkiye                                                                  158
Panama (Republic of)                                                                 120
Malta                                                                                 70
Liberia (Republic of)                                                                 54
Romania                                                                               47
Marshall Islands (Republic of the)                                                    47
Bulgaria (Republic of)                                                                36
Ukraine                                                                               26
Comoros (Union of the)                                                                21
Sierra Leone                                                                 

In [36]:
import pandas as pd
pd.option_context('display.max_rows', None)

In [57]:
ais_enriched3["mmsi"].nunique()

1177

In [56]:
# fonction that assigns a destination country
ais_enriched3, destination_ports = indic.assign_destination_country(ais_enriched2)

Percentage missing destination port
0.06966864910790145
Number missing destination port
82
mmsi
207008000    [POTI, VARNA, NECOCHEA, CHORNOMORSK, CHERNOMORSK]
207037000    [VARNA, SMEDEREVO, VARNA     ORAGE, JP SGM NAK...
207038000                              [OPENSEA, PP#23, VARNA]
207094000                                               [None]
207095000                                               [None]
                                   ...                        
677075600    [KHERSON/UKRAINE, BANDIRMA/TURKEY, LULA FIELD,...
677080700                  [ROSTOV / RUSSIA, BANDIRMA, RU ROV]
677085700                           [RASA, ALEXANDRIA, UA NIK]
677092400                  [BANGHAZI / LIBYA, 0, JP OIT, None]
677096200                      [MISURATA, NLKAM10084HRARE0001]
Name: destination, Length: 1177, dtype: object
Classification of destination countries
Turkey                                           114
Russia                                            90
Romania           

In [347]:

def check_distance_levenshtein(data: pd.DataFrame, variables=None) -> pd.DataFrame:
    """
    Compute levenshtein distance between list of columns of a Pandas DataFrame
    Parameters
    ----------
    data: Pandas dataframe with some columns to be compared
    variables: List of variables from which distance will be computed. The first one will be used as benchmark
    Returns
    -------
    Initial dataframe with new columns representing levenshtein ratio (computed using :meth:`rapidfuzz.fuzz.partial_ratio`)
    between variables
    """

    if variables is None:
        variables = ['libel_clean_relevanc', 'libel_clean_OFF', 'libel_clean_IRI']

    data[variables] = data[variables].astype(str)
    newvars = ["ratio{}".format(str(i)) for i in range(1, len(variables))]
    data[newvars] = pd.concat(
        [data.apply(lambda x: rapidfuzz.fuzz.partial_ratio(x[variables[0]], x[y]), axis=1) for y in variables[1:]],
        axis=1
    )
    return data


In [358]:
import numpy as np
def fuzzy_match_destination_country(ais_enriched):
    ais_enriched3["destination_c"] = (ais_enriched["destination"].replace("None", np.nan)
                        .replace('B 2 A', np.nan)
                        .replace('J0', np.nan)
                        .replace('0', np.nan)
                        .replace(',',np.nan)
                        .replace('G', np.nan)
                        .replace('I0', np.nan)
                        .replace('C', np.nan)
                        .replace('D', np.nan)
                        .replace('C0', np.nan)
                        .replace('S0', np.nan).replace('M', np.nan)
                        .replace('X', np.nan).replace('A', np.nan)
                        .replace('ZRP', np.nan)
                        .replace('AT SEA', np.nan)
                        .replace("$", np.nan)
                        .replace('#HDCR*H!!!)JM0I*!&', np.nan)
                        .replace("%", np.nan)
                        .replace("-", np.nan).replace('-C>-',np.nan)
                        .replace('-C?M',np.nan)
                        .replace('-[C?<)RD\\K$OQ8)',np.nan)
                        .replace( '.',np.nan)
                        .replace( '...',np.nan)
                        .replace('0XA F3L86F,[E^CON',np.nan)
                        .replace('17 B#',np.nan)
                        .replace('2',np.nan)
                        .replace('3',np.nan)
                        .replace('450',np.nan)
                        .replace(r'(\s\/\s[a-zA-Z]+)', '', regex=True)
                        .replace(r'(\/[a-zA-Z]+)', '', regex=True)
                        .replace('FISHING AREA', np.nan))
    list_errors = ['AREA3 FISHING GROUND', 'ARM GAURD ON BOARD',
 'ARM GUARD ON BOARD', 'ARM.GUARDS ON BOARD', 'ARMED GUARD ON BOARD', 'ARMED GUARDS ON BOAP',
 'ARMED GUARDS ON BOAR', 'ARMED GUARDS ONB', 'ARMED GUARDS ONBOARD', 'ARMED ON BOARD',
 'ARMGUARD O.B', 'ARMGUARD ONBOARD', 'ARMS GUARDS ON BOARD']
    ais_enriched3.loc[ais_enriched["destination_c"].isin(list_errors), "destination_c"] = np.nan
    ais_enriched3["length_dest"] = ais_enriched3["destination_c"].str.len()
    ais_enriched3.loc[ais_enriched3["length_dest"] == 2, "destination_c"] = np.nan
    print("Number of unique destination ports")
    print(ais_enriched3["destination_c"].nunique())
    print("shape of ais")
    print(ais_enriched3.shape)
    # import port data
    ports = pd.read_csv('https://msi.nga.mil/api/publications/download?type=view&key=16920959/SFH00000/UpdatedPub150.csv')
    ports = ports.loc[:, ["Main Port Name", "Country Code"]]
    ports["Main Port Name"] = ports["Main Port Name"].str.upper()
    ports = ports.rename({"Main Port Name" : "destination"}, axis=1)
    # filter boats with destination ports
    ais_enriched4 = ais_enriched3.loc[~ais_enriched3["destination_c"].isna(),:]
    print("shape of ais filtered with no na destination")
    print(ais_enriched4.shape)
    
    # select one destination by boat
    destination_ais = ais_enriched4.groupby("mmsi")["destination_c"].first()
    destination_ais = destination_ais.reset_index().copy()
    cart_prod = destination_ais.merge(ports, how='cross')
    # fuzzy matching using rapid fuzz
    print("Fuzzy matching in process")
    res_fuz_match = check_distance_levenshtein(cart_prod, ["destination_c", "destination"])
    idx = res_fuz_match.groupby(['destination_c'])['ratio1'].transform(max) == res_fuz_match['ratio1']
    res_fuz_match2 = res_fuz_match[idx]
    res_fuz_match2["true_match"] = np.nan
    res_fuz_match2.loc[res_fuz_match2["ratio1"]>80, "true_match"] = \
        res_fuz_match2.loc[res_fuz_match2["ratio1"]>80, "destination"]
    return ais_enriched4, res_fuz_match2

In [359]:
ais_enriched4, res_fuz_match2 = fuzzy_match_destination_country(ais_enriched3)

Number of unique destination ports
4003
shape of ais
(3333279, 131)
shape of ais filtered with no na destination
(3047816, 131)
Fuzzy matching in process


/tmp/ipykernel_1585/16609960.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_fuz_match2["true_match"] = np.nan


In [360]:
res_fuz_match2.head()

,mmsi,destination_c,destination,Country Code,ratio1,true_match
1447,207008000,POTI,POTI,Georgia,100.000000,POTI
5150,207037000,VARNA,VARNA,Bulgaria,100.000000,VARNA
7037,207038000,OPENSEA,ODENSE,Denmark,83.333333,ODENSE
10910,207133000,VARNA,VARNA,Bulgaria,100.000000,VARNA
13790,207228000,VARNA,VARNA,Bulgaria,100.000000,VARNA


In [365]:
res_fuz_match2.loc[res_fuz_match2["ratio1"]<80, :].sort_values("ratio1", ascending=False).head(20)

,mmsi,destination_c,destination,Country Code,ratio1,true_match
522536,264900331,HARBOUR TOWAGE CTA,KEY HARBOUR,Canada,77.777778,NaN
1672578,273395260,CUXHAVEN,WILHELMSHAVEN,Germany,76.923077,NaN
1032357,273315430,RAVENNA,BREMERHAVEN,Germany,76.923077,NaN
2573940,518857000,CHALKIS,KYLE OF LOCHALSH,United Kingdom,76.923077,NaN
851246,271072367,MINDELO,MIDDELFART,Denmark,76.923077,NaN
2328743,371202000,RAVENNA,ANTSIRANANA,Madagascar,76.923077,NaN
2328357,371202000,RAVENNA,BREMERHAVEN,Germany,76.923077,NaN
2327792,371202000,RAVENNA,CASTLETOWN BEARHAVEN,Ireland,76.923077,NaN
1671485,273395260,CUXHAVEN,NORTH HAVEN,United States,76.923077,NaN
1031792,273315430,RAVENNA,CASTLETOWN BEARHAVEN,Ireland,76.923077,NaN
